In [164]:
# importing dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [165]:
# importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [166]:
# loading the csv data to a pandas dataframe
heart_data = pd.read_csv('/content/heart.csv')

In [167]:
# print the first 5 rows of the dataset
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [168]:
# Printingn the shape of the dataset
heart_data.shape

(303, 14)

In [169]:
# Checking for missing values
heart_data.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [170]:
# Checking the distribution of the target variable
heart_data['target'].value_counts()

,count
target,
1,165
0,138


In [171]:
# Splitting the features and the target
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [172]:
# Convert both varuables to numpy arrays
X = np.asarray(X)
Y = np.asarray(Y)

###Model Selection

In [173]:
# list all the models that were imported for the sake of comparing them with their default parameters
models = [LogisticRegression(max_iter=10000), KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(random_state=0), SVC(kernel='linear')]

In [174]:
# write a function to compare the models
def compare_models_cross_validation():
 for model in models:
  cv_score = cross_val_score(model, X, Y, cv=5)
  mean_accuracy = sum(cv_score)/len(cv_score)
  mean_accuracy = mean_accuracy*100
  mean_accuracy = round(mean_accuracy, 2)
  print('Cross Validation accuracies for the ', model, ' = ', cv_score)
  print('Accuracy % of the ', model, mean_accuracy)
  print("------------------------------------")

In [175]:
# call the function
compare_models_cross_validation()

Cross Validation accuracies for the  LogisticRegression(max_iter=10000)  =  [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy % of the  LogisticRegression(max_iter=10000) 82.83
------------------------------------
Cross Validation accuracies for the  KNeighborsClassifier()  =  [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy % of the  KNeighborsClassifier() 64.39
------------------------------------
Cross Validation accuracies for the  DecisionTreeClassifier()  =  [0.75409836 0.83606557 0.70491803 0.73333333 0.71666667]
Accuracy % of the  DecisionTreeClassifier() 74.9
------------------------------------
Cross Validation accuracies for the  RandomForestClassifier(random_state=0)  =  [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy % of the  RandomForestClassifier(random_state=0) 83.81
------------------------------------
Cross Validation accuracies for the  SVC(kernel='linear')  =  [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667

In [176]:
# Creating an object that contains hyperparameters values for the above mentioned models
models_hyperparameters = {
    'log_reg_hyperparameters': {
        'C' : [1,5,10,20]
    },
      "KNN_hyperparameters" : {
        'n_neighbors' : [3,5,10]
    },
      "decision_tree_hyperparameters": {
    'criterion': ['gini', 'entropy', 'log_loss'],  # metric to measure split quality
    'max_depth': [None, 5, 10, 20, 30],            # max depth of tree
    'min_samples_split': [2, 5, 10],               # min samples to split a node
    'min_samples_leaf': [1, 2, 4],                 # min samples at a leaf node
    'max_features': [None, 'sqrt', 'log2'],        # number of features to consider
    'ccp_alpha': [0.0, 0.01, 0.05]                 # pruning strength (post-pruning)
    },

     "random_forest_hyperparameters" : {
        'n_estimators' : [10, 20, 50, 100]
    },
    "svc_hyperparameters": {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'C' : [1,5,10,20]
    },

}

In [177]:
# Check the type of object we have
type(models_hyperparameters)

dict

In [178]:
# print the model hyperparameters keys
models_hyperparameters.keys()

dict_keys(['log_reg_hyperparameters', 'KNN_hyperparameters', 'decision_tree_hyperparameters', 'random_forest_hyperparameters', 'svc_hyperparameters'])

In [179]:
# load the hyperparameter keys into a list and store into model_keys
model_keys = list(models_hyperparameters.keys())

In [180]:
# print the model hyperparameters only of a single model using the list modelkeys
models_hyperparameters[model_keys[0]]

{'C': [1, 5, 10, 20]}

In [181]:
def ModelSelection(list_of_models, hyperparameters_dictionary):

  result = []

  i = 0

  for model in list_of_models:

    key = model_keys[i]

    params = hyperparameters_dictionary[key]

    i += 1

    print(model)
    print(params)
    print('---------------------------------')


    classifier = GridSearchCV(model, params, cv=5)

    # fitting the data to classifier
    classifier.fit(X,Y)

    result.append({
        'model used' : model,
        'highest score' : classifier.best_score_,
        'best hyperparameters' : classifier.best_params_
    })

  result_dataframe = pd.DataFrame(result, columns = ['model used','highest score','best hyperparameters'])

  return result_dataframe

In [182]:
ModelSelection(models, models_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
---------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
---------------------------------
DecisionTreeClassifier()
{'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 5, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': [None, 'sqrt', 'log2'], 'ccp_alpha': [0.0, 0.01, 0.05]}
---------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
---------------------------------
SVC(kernel='linear')
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
---------------------------------


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
2,DecisionTreeClassifier(),0.828197,"{'ccp_alpha': 0.01, 'criterion': 'gini', 'max_..."
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
4,SVC(kernel='linear'),0.828306,"{'C': 1, 'kernel': 'linear'}"


So we would be building a random forest model with n_estimators = 100 as it has the highest accuracy.